In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.optim as optim
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence

from torch.utils.data import DataLoader
import glob

!pip install pydot

2023-08-02 12:43:29.303840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 12:43:29.438720: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-02 12:43:29.468513: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-02 12:43:30.168292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

#### Actual Data

In [3]:
## generate test / train split

class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, seq_len, batch_size):
        self.x, self.y = x_set, y_set
        self.seq_len = seq_len
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.seq_len)))

        
    def most_common(self,lst):
        # print("mc",lst)
        lst = [np.where(r==1)[0][0] for r in lst]
        return (np.bincount(lst).argmax())
    
    def __getitem__(self, idx):
        start_idx = idx * self.seq_len
        end_idx = (idx + 1) * self.seq_len

        batch_x = self.x[start_idx:end_idx]
        batch_y = self.y[start_idx:end_idx]
        
        
        # most_com = self.most_common(batch_y)
        # print(most_com)
        
        batch_x = torch.from_numpy(batch_x)
        batch_y = torch.from_numpy(batch_y)
        
        # Pad sequences to ensure they have the same length within the batch
        pad_len = self.seq_len - batch_x.shape[0]
        if pad_len > 0:
            pad_shape = (pad_len,) + batch_x.shape[1:]
            pad_shape_y = (pad_len,) + batch_y.shape[1:]
            
            batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
            batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)


        return batch_x, batch_y


    def on_epoch_end(self):
        indices = np.arange(len(self.x))
        np.random.shuffle(indices)
        self.x = self.x[indices]
        self.y = self.y[indices]
        
def generate_data_split(subject_id):
    # Get CSV files list from a folder
    train_path = './Train'
    test_path = './Test'
    
    csv_path = './ProcessedDatasets/Knot_Tying'
    
    csv_files = glob.glob(csv_path + "/*.csv")
    
    train_df_list = []
    test_df_list = []
    
    for file in csv_files:
        if(subject_id in file):
            test_df_list.append(pd.read_csv(file))
#             print(file)
        else:
            train_df_list.append(pd.read_csv(file))
            

    print('Train Subject Trials: ',len(train_df_list))
    print('Test Subject Trials: ',len(test_df_list))
    
    # Concatenate all DataFrames
    train_df   = pd.concat(train_df_list, ignore_index=True)
    test_df   = pd.concat(test_df_list, ignore_index=True)

    
    lb = preprocessing.LabelBinarizer()

    train_labels= train_df.pop('label')
    train_features = train_df

    test_labels= test_df.pop('label')
    test_features = test_df


    all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
    lb.fit(all_class_names)

    train_labels = lb.transform(train_labels)
    test_labels = lb.transform(test_labels)
    
    train_x = train_features.to_numpy()
    train_y = train_labels

    test_x = test_features.to_numpy()
    test_y = test_labels

    seq_len = 30
    batch_size = 64

    valid_test_split = 0.8
    # Step 2: Split the remaining data into validation and test sets
    val_x, test_x, val_y, test_y = train_test_split(
    test_x, test_y, test_size=valid_test_split, random_state=42)
    
    train_dataset = CustomDataset(train_x, train_y, seq_len, batch_size)
    
    val_dataset = CustomDataset(val_x, val_y, seq_len, batch_size)
    
    test_dataset = CustomDataset(test_x, test_y, seq_len, batch_size)

    train_dataloader = tf.data.Dataset.from_generator(
        lambda: train_dataset,
        output_signature=(
            tf.TensorSpec(shape=( seq_len, train_x.shape[1]), dtype=tf.float32),
            tf.TensorSpec(shape=( seq_len, train_y.shape[1]), dtype=tf.float32),
        ),
    # )
    ).repeat()
    train_dataloader = train_dataloader.batch(batch_size)


    val_dataloader = tf.data.Dataset.from_generator(
        lambda: val_dataset,
        output_signature=(
            tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
            tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
        ),
    )
    val_dataloader = val_dataloader.batch(batch_size)
    

    test_dataloader = tf.data.Dataset.from_generator(
        lambda: test_dataset,
        output_signature=(
            tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
            tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
        ),
    )
    test_dataloader = test_dataloader.batch(batch_size)
    
    return train_dataloader, val_dataloader, test_dataloader
    
subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']

for subject in subjects:
    train_dataloader, val_dataloader, test_dataloader = generate_data_split(subject)
    train_features, train_labels = next(iter(val_dataloader))

    input_shape = train_features.shape[1:]
    output_dim = train_labels.shape[2]

    print("input_dim: ",input_shape)
    print("output_dim: ",output_dim)
    
    break



Train Subject Trials:  32
Test Subject Trials:  4
input_dim:  (30, 66)
output_dim:  14


2023-08-02 12:43:32.866311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-02 12:43:32.866937: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-02 12:43:32.867011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-02 12:43:32.867455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

### LSTM

In [4]:
def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)
    
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D(data_format="channels_first", keepdims=True)(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


train_dataloader, val_dataloader, test_dataloader = generate_data_split('S04')
train_features, train_labels = next(iter(train_dataloader))

input_shape = train_features.shape[1:]
output_dim = train_labels.shape[2]

print("input_dim: ",input_shape)
print("output_dim: ",output_dim)


model = make_model(input_shape=input_shape, num_classes=output_dim)
# keras.utils.plot_model(model, show_shapes=True)
# model.summary()


Train Subject Trials:  31
Test Subject Trials:  5
input_dim:  (30, 66)
output_dim:  14


### Train and Validation

In [5]:
epochs = 25
steps_per_epoch = 500

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)




subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']


model.save_weights('./checkpoints/blank_state')

for subject in subjects:
    
    print(" *** --------START TRAIN--------- *** ")
    print("SUBJECT EXCLUDED: ",subject)
    
    train_dataloader, val_dataloader, test_dataloader = generate_data_split(subject)

    model.load_weights('./checkpoints/blank_state')

    history = model.fit(
        train_dataloader,
        epochs=epochs,
        callbacks=callbacks,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_dataloader,
        verbose=1,
    )

    print(" *** -------END TRAIN-------- *** ")
    print(" *** -------START EVAL-------- *** ")
    
    model.evaluate(test_dataloader, verbose=1)

    print(" *** -------END EVAL-------- *** ")
    




 *** --------START TRAIN--------- *** 
SUBJECT EXCLUDED:  S02
Train Subject Trials:  32
Test Subject Trials:  4
Epoch 1/25


2023-08-02 12:43:46.139943: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 45.15M (47341056 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-08-02 12:43:46.139997: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.47MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-08-02 12:43:46.140010: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2023-08-02 12:43:46.201094: E tensorflow/stream_executor/cuda/cuda_dnn.cc:389] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-08-02 12:43:46.201156: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES fai

UnimplementedError: Graph execution error:

Detected at node 'model/conv1d/Conv1D' defined at (most recent call last):
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/kesharaw/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 195, in start


In [16]:
model.evaluate(test_dataloader, verbose=1)


      1/Unknown - 0s 20ms/step - loss: 7.4180 - categorical_accuracy: 0.2594

3/3 [==============================] - 0s 9ms/step - loss: 7.4923 - categorical_accuracy: 0.2656


[7.492281436920166, 0.2655773460865021]

### Train and Evaluate

In [ ]:
# input_shape = (30,66)
# input_shape = train_features.shape

# train_features = train_features.to_numpy()
# train_features = train_features.reshape((train_features.shape[0], train_features.shape[1], 1))
# input_shape = train_features.shape
train_dataloader, test_dataloader = generate_data_split(subject)
train_features, train_labels = next(iter(train_dataloader))

input_shape = train_features.shape[1:]
output_dim = train_labels.shape[2]

print("input_dim: ",input_shape)
print("output_dim: ",output_dim)



model = build_model(
    input_shape,
    output_dim,
    head_size=256,
    num_heads=2,
    ff_dim=512,
    num_transformer_blocks=4,
    mlp_units=[256,128],
    mlp_dropout=0.5,
    dropout=0.3,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["categorical_accuracy"],
)
# # # # model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

epochs = 50
# steps_per_epoch = train_dataloader.__len__()//1
# print("steps_per", steps_per_epoch)


subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']

for subject in subjects:
    
    print(" *** --------START--------- *** ")
    print("SUBJECT EXCLUDED: ",subject)
    
    train_dataloader, test_dataloader = generate_data_split(subject)

    model.fit(
        train_dataloader,
        epochs=epochs,
        callbacks=callbacks,
        steps_per_epoch=500
#         validation_data = test_dataloader
    )


    print(" *** -------END-------- *** ")
    



## Pytorch dataloader implementation

# def gen(torch_loader):
#     for x,y in torch_loader:
#         yield (x,y)

# train = gen(train_dataloader)

# epochs = 300
# steps_per_epoch = train_dataloader.__len__()//5
# print("steps_per", steps_per_epoch)

# model.fit(
#     train,
#     epochs=epochs,
#     callbacks=callbacks,
#     steps_per_epoch=steps_per_epoch
# )

# # model.evaluate(x_test, y_test, verbose=1)


Train Subject Trials:  32
Test Subject Trials:  4
input_dim:  (30, 66)
output_dim:  14
14
 *** --------START--------- *** 
SUBJECT EXCLUDED:  S02
Train Subject Trials:  32
Test Subject Trials:  4
Epoch 1/50
500/500 [==============================] - 11s 17ms/step - loss: 8.4678 - categorical_accuracy: 0.2082
Epoch 2/50
500/500 [==============================] - 9s 17ms/step - loss: 2.1732 - categorical_accuracy: 0.2531
Epoch 3/50
500/500 [==============================] - 8s 17ms/step - loss: 1.9199 - categorical_accuracy: 0.2687
Epoch 4/50
500/500 [==============================] - 8s 17ms/step - loss: 1.7385 - categorical_accuracy: 0.2889
Epoch 5/50
500/500 [==============================] - 8s 17ms/step - loss: 1.6053 - categorical_accuracy: 0.3186
Epoch 6/50
500/500 [==============================] - 8s 17ms/step - loss: 1.4901 - categorical_accuracy: 0.3880
Epoch 7/50
500/500 [==============================] - 8s 17ms/step - loss: 1.4061 - categorical_accuracy: 0.4259
Epoch 8/50
5

In [85]:
# !pip install -q -U keras-tuner



[notice] A new release of pip available: 22.2.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [91]:
# import keras_tuner as kt

# train_features, train_labels = next(iter(train_dataloader))

# input_shape = train_features.shape[1:]
# output_dim = train_labels.shape[2]

# print("input_dim: ",input_shape)
# print("output_dim: ",output_dim)


# def model_builder(hp):
    
    
#     hp_headsize = hp.Int('units', min_value=16, max_value=512, step=16)
#     hp_numheads = hp.Int('units', min_value=2, max_value=16, step=2)
#     hp_ff_dim = hp.Int('units', min_value=1, max_value=512, step=16)
#     hp_num_transformer_blocks = hp.Int('units', min_value=1, max_value=16, step=2)

#     model = build_model(
#     input_shape,
#     output_dim,
#     head_size=hp_headsize,
#     num_heads=hp_numheads,
#     ff_dim=hp_ff_dim,
#     num_transformer_blocks=hp_num_transformer_blocks,
#     mlp_units=[128],
#     mlp_dropout=0.4,
#     dropout=0.25,
#     )

#     model.compile(
#         loss="categorical_crossentropy",
#         optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#         metrics=["categorical_accuracy"],
#     )
    
#     return model


# tuner = kt.Hyperband(model_builder,
#                      objective='val_accuracy',
#                      max_epochs=10,
#                      factor=3,
#                      project_name='intro_to_kt')


input_dim:  (30, 71)
output_dim:  14
INFO:tensorflow:Reloading Tuner from ./intro_to_kt/tuner0.json


In [92]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
# tuner.search(train_dataloader, epochs=10, steps_per_epoch=500, callbacks=[stop_early])

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)


In [35]:
model.evaluate(test_dataloader, verbose=1)

# test_features, test_labels = next(iter(test_dataloader))

# y_pred = model(test_features)

# relu = nn.ReLU()


# for idx,y in enumerate(y_pred) :
#     for i,sample in enumerate(y):
#         print(np.argmax(test_labels[idx][i].numpy()))
#         print(np.argmax(sample))
#         print('-----')
        
#     break

5/5 [==============================] - 0s 14ms/step - loss: 0.9062 - categorical_accuracy: 0.6734


[0.9061856865882874, 0.6734204888343811]

In [101]:
model.save_weights('./checkpoints/progress_v2')

In [117]:
# Restore the weights
model.load_weights('./checkpoints/progress_v2')


ValueError: Received incompatible tensor with shape (71,) when attempting to restore variable with shape (66,) and name layer_with_weights-0/beta/.ATTRIBUTES/VARIABLE_VALUE.